In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

insurance_df = pd.read_csv('./TravelInsurancePrediction.csv')
insurance_df = insurance_df[['Age',	'Employment Type',	'GraduateOr0t'	,	'FamilyMembers',	'ChronicDiseases',	'FrequentFlyer',	'EverTravelledAbroad',	'AnnualIncome1','TravelInsurance']]
insurance_df.head()

,Age,Employment Type,GraduateOr0t,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,AnnualIncome1,TravelInsurance
0,31,0,1,6,1,0,0,40,0
1,31,1,1,7,0,0,0,125,0
2,34,1,1,4,1,0,0,50,1
3,28,1,1,3,1,0,0,70,0
4,28,1,1,8,1,1,0,70,0


In [12]:
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987 entries, 0 to 1986
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Age                  1987 non-null   int64
 1   Employment Type      1987 non-null   int64
 2   GraduateOr0t         1987 non-null   int64
 3   FamilyMembers        1987 non-null   int64
 4   ChronicDiseases      1987 non-null   int64
 5   FrequentFlyer        1987 non-null   int64
 6   EverTravelledAbroad  1987 non-null   int64
 7   AnnualIncome1        1987 non-null   int64
 8   TravelInsurance      1987 non-null   int64
dtypes: int64(9)
memory usage: 139.8 KB


In [13]:
insurance_df['TravelInsurance'].value_counts()

0    1277
1     710
Name: TravelInsurance, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

def get_preprocessed_df(df=None):
    df_copy = df.copy()
    return df_copy

In [15]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수.
def get_train_test_dataset(df=None):
    # 인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
    df_copy = get_preprocessed_df(df)
    # DataFrame의 맨 마지막 컬럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 층화추출 기반 균등분할
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=y_target)
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

In [16]:
X_train, X_test, y_train, y_test = get_train_test_dataset(insurance_df)

In [17]:
print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.shape[0] * 100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0] * 100)

학습 데이터 레이블 값 비율
0    64.244604
1    35.755396
Name: TravelInsurance, dtype: float64
테스트 데이터 레이블 값 비율
0    64.321608
1    35.678392
Name: TravelInsurance, dtype: float64


In [18]:
# 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [19]:
# 인자로 사이킷런의 Estimator객체와, 학습/테스트 데이터 세트를 입력 받아서 학습/예측/평가 수행.
# ML알고리즘 객체 전달 후 학습/예측/평가 함수
import time
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    start_time = time.time()
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)
    print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

In [23]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
get_model_train_eval(rf_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[339  45]
 [ 84 129]]
정확도: 0.7839, 정밀도: 0.7414, 재현율: 0.6056, F1: 0.6667
수행 시간: 2.7 초 
